## Generate elog entries for Stanley lab TOI discrete samples from underway (Endeavor)

In [1]:
import os
import pandas as pd

# some configuration

# your paths will vary
DIR=r'D:\nes-lter-ims-test-data'

EN608='En608'
EN617='En617'

# column headers
INSTRUMENT = 'Instrument'
ACTION = 'Action'
COMMENTS = 'Comment'
STATION = 'Station'
CAST = 'Cast'
DATETIME = 'dateTime8601'
PUMP_TYPE = 'pump_type'
LAT='Latitude'
LON='Longitude'

# values
TOI_DISCRETE = 'TOI discrete'
UNDERWAY_SCIENCE_SEAWATER = 'Underway Science seawater'
USS_IMPELLER = 'Underway Science seawater impeller'
USS_DIAPHRAGM = 'Underway Science seawater diaphragm pump'

IMPELLER_PUMP = 'impeller pump'
DIAPHRAGM_PUMP = 'diaphragm pump'

Parse Stanley-lab-provided columnar text file. This contains date/time/lat/lon and pump type

In [2]:
cruise=EN617

FILE = '{}_TOI_underwaysampletimes.txt'.format(cruise)
path = os.path.join(DIR, FILE)
assert os.path.exists(path)

df = pd.read_csv(path, sep='\s\s+')
df.columns = [DATETIME, PUMP_TYPE, LAT, LON, 'ignore']
df.pop('ignore')
# add the action column
df[ACTION] = TOI_DISCRETE
# parse datetimes
df[DATETIME] = pd.to_datetime(df[DATETIME])
df.head()

C:\Users\joefutrelle\Anaconda2\envs\nes-lter-ims-dev\lib\site-packages\ipykernel\__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,dateTime8601,pump_type,Latitude,Longitude,Action
0,2018-07-20 23:14:00,0,41.032503,-70.878400,TOI discrete
1,2018-07-20 23:15:00,0,41.032690,-70.878290,TOI discrete
2,2018-07-21 00:32:00,0,41.025502,-70.663102,TOI discrete
3,2018-07-21 00:48:00,0,41.029698,-70.720297,TOI discrete
4,2018-07-21 02:04:00,0,41.030904,-70.840800,TOI discrete


In [3]:
log = df.copy()

# for EN608, the instrument is "Underway Science seawater" and information about
# pump type is put into the comment field
if cruise == EN608:
    log[INSTRUMENT] = UNDERWAY_SCIENCE_SEAWATER
    log[COMMENTS] = log[PUMP_TYPE].replace({
        0: IMPELLER_PUMP,
        1: DIAPHRAGM_PUMP
    })

# For EN617, the instrument is "Underwater Science seawater impeller" or
# "Underway Science seawater diaphragm pump" depending on the pump type.
# leave comment field blank
elif cruise == EN617:
    log[INSTRUMENT] = log[PUMP_TYPE].replace({
        0: USS_IMPELLER,
        1: USS_DIAPHRAGM,
    })
    log[COMMENTS] = ''

# add placeholder columns for station and cast (both are n/a to these events)
log[STATION] = ''
log[CAST] = ''

# keep only the columns we want
log = log[[DATETIME, INSTRUMENT, ACTION, STATION, CAST, LAT, LON, COMMENTS]]
log.head()

,dateTime8601,Instrument,Action,Station,Cast,Latitude,Longitude,Comment
0,2018-07-20 23:14:00,Underway Science seawater impeller,TOI discrete,,,41.032503,-70.878400,
1,2018-07-20 23:15:00,Underway Science seawater impeller,TOI discrete,,,41.032690,-70.878290,
2,2018-07-21 00:32:00,Underway Science seawater impeller,TOI discrete,,,41.025502,-70.663102,
3,2018-07-21 00:48:00,Underway Science seawater impeller,TOI discrete,,,41.029698,-70.720297,
4,2018-07-21 02:04:00,Underway Science seawater impeller,TOI discrete,,,41.030904,-70.840800,


In [4]:
# write output to a file
OUTPUT_DIR = os.path.join(DIR, 'output')

out_path = os.path.join(OUTPUT_DIR, '{}_toi_elog.csv'.format(cruise).lower())

log.to_csv(out_path, index=None)